In [1]:
#!pip install arabic-stopwords # not useful


In [ ]:
#import os
#os.kill(os.getpid(), 9)


In [1]:
!pip install pytorch-lightning
!pip install torchmetrics



In [2]:
import numpy as np
import pandas as pd
import transformers
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import  AutoModel,AutoTokenizer
import pytorch_lightning as pl
from torchmetrics import F1Score

#from torchmetrics.functional import f1
seed = pl.seed_everything(42)
#import arabicstopwords.arabicstopwords as ast
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
import torchmetrics
from torch.optim import AdamW#
from transformers import AutoModel, get_scheduler
import pytorch_lightning as pl
import torch
from torch import nn
from sklearn.metrics import f1_score, make_scorer


INFO:lightning_fabric.utilities.seed:Seed set to 42


In [3]:
# setting paddas for better view for our texts
pd.set_option('display.max_colwidth',None)

In [4]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.auto import tqdm
TrainData_name = 'All targets-Train.csv'
TestData_name = 'All targets-Test.csv'



!gdown --id 1-6gk5tyS3W1PcsAqfJu46mqSZtRjpyr4
!gdown --id 1-20B-rvHasK5GZF6ASlp7SVREWlBet3u

!ls


train_df = pd.read_csv(TrainData_name)
test_df = pd.read_csv(TestData_name)
##
train_df.head()
#train_df = pd.read_csv("V2_Mawqif_AllTargets_Train.csv")
print(train_df.shape)
print(test_df.shape)

# Filter rows based on the selected target
#train_df = train_df[train_df['target'] == selectedTarget]
#test_df = test_df[test_df['target'] == selectedTarget]

# Print the shapes of the datasets
print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")

stance_mapping = {'Favor': 1, 'Against': 2, 'None': 0}

train_df['stance'] = train_df['stance'].fillna('None')
test_df['stance'] = test_df['stance'].fillna('None')

train_df['stance_encoded'] = train_df['stance'].map(stance_mapping)
y_train = train_df['stance'].map(stance_mapping).values
train_df['stance'] = train_df['stance'].map(stance_mapping).values
# Checking the number of features in the embeddings
#print(f"Number of features in the embeddings: {train_embeddings.shape[1]}")
train=train_df[['ID','text','stance']]
test_df = test_df[['ID','text','stance']]
test_df['stance'] = test_df['stance'].map(stance_mapping).values



/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-6gk5tyS3W1PcsAqfJu46mqSZtRjpyr4
To: /content/All targets-Train.csv
100% 3.92M/3.92M [00:00<00:00, 39.8MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-20B-rvHasK5GZF6ASlp7SVREWlBet3u
To: /content/All targets-Test.csv
100% 691k/691k [00:00<00:00, 95.5MB/s]
'All targets-Test.csv'	'All targets-Train.csv'   lightning_logs   model_checkpoints   sample_data
(3502, 41)
(619, 41)
Training set shape: (3502, 41)
Test set shape: (619, 41)


In [6]:
#stop_words = ast.stopwords_list() مالها فايدة

# a small function to remove stop words
#def remove_stop_words(text):
   # return ' '.join(word for word in text.split() if word not in stop_words)

#train['text'] = train['text'].apply(remove_stop_words)
#test_df['text'] = test_df['text'].apply(remove_stop_words)



In [5]:
#train_df.describe()
train
#y_train

,ID,text,stance
0,1,@DaringQueenz عشان يلمع صورته ويعنني تمكين المرأة ويصير ترند والحكومة هي اكثر من تقمع المرأة اخر شيء كل الي فرحانين بالقرار ودارسين قانون متوظفين كاشير في مول راتبهم 3 الاف,2
1,3,@Ask_Makkah_ روح حلل محد يم تطعيم كورونا شف الحرم البارح مليونين بدون شرط تحصين😅,0
2,4,هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة فكرية تغزو عقول بعض الرجال حيث إنه إذ رأى تمكين للمرأة حتى ولو كان بسيطًا فهو من المرجح بالطبع ربطه للموضوع مع - النسوية - وأتهام الفكرة بأكملها بالشيطنة والخبث . URL,1
3,6,#LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في المجال التقني ومن خلاله السعودية تستعرض قدرتها في التحول الرقمي اللي تشهدها كافة قطاعاتها 👌🏼 URL,1
4,7,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن لا ينبغي أن تكون ذريعة لسد الطريق امام التحول التقني لأننا نجهل كيف نحمي بياناتنا في هذا الفضاء التقني. بل أن عملية التحول الرقمي ليست خيارًا نقبله أو نرفضه ونحن قطاع يتسم بالشفافية والمشاركة مع المجتمع وأصحاب المصلحة.,1
...,...,...,...
3497,4117,يا جماعة الخير انا و اهلي حاشتنا كورونا خلصنا من العام الماضي و مطعمين تطعيم كوفيد 19 ليش لازم نلتزم بالحظر المفروض كرت التطعيم بدال الهويه يخلونا نمارس حياتنا .@Drbaselalsabah مايصير حالنا حال الجميع .,0
3498,4118,@omanilawyer999 للاسف بعض مدراء الدوائر من العقليات القديمة سبب في تعثر التحول الالكتروني للموسسات.,1
3499,4119,ليس عصر تمكين المرأة . وإنما عصر تدمير الأسرة والمجتمع .ولكن غباء بعض النساء لايشعرن بذلك . بسبب نشوة الإنتصار الكاذب على الرجل .متى تعي المرأة المسلمة أن الهدف ليست هي وإنما مجتمعها كله ؟,2
3500,4120,ياجماعه الخير هل يعقل اليوم جهة خدمية تذهب لانجاز معاملة تجد الرد ان السيستم عطلان من يومين وفوقهم اجازة نهاية الاسبوع نحن اليوم نتكلم عن الخدمات و التحول الالكتروني ومازلنا في عزلة عن الغير اقرار القرارات سهل على ارض ترى العكس,1


In [8]:
#!git clone https://github.com/aub-mind/arabert/
#!pip install farasapy

In [9]:
#from arabert.preprocess import ArabertPreprocessor # also not useful
#arabert_prep = ArabertPreprocessor(model_name='aubmindlab/bert-base-arabertv02')#'aubmindlab/bert-base-arabertv02'.  aubmindlab/bert-base-arabert

Prepare data to the model

In [6]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train, test_size=0.1, stratify=train['stance'])

In [7]:
class MyDataset(Dataset):
  def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: AutoTokenizer,
      text_preprocessor=None,
      max_token_len: int = 128
    ):
    self.data = data
    self.tokenizer = tokenizer
    self.text_preprocessor = text_preprocessor
    self.max_token_len = max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    text = data_row.text
    if self.text_preprocessor is not None:
      text = self.text_preprocessor.preprocess(text)

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_token_len,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True, # to make sure each sequence is maximum of max length
      return_tensors='pt', #return it as pytorch
    )

    return dict(
      text=text,
      input_ids=encoding["input_ids"].flatten(), ##we use flatten to remove X dimension
      attention_mask=encoding["attention_mask"].flatten(),
      labels = torch.tensor(data_row.stance, dtype=torch.long)
    )

In [8]:
test_labels = []
test_pred = []
probsoft = []

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchmetrics
from transformers import AutoModel, AutoTokenizer, AdamW, get_scheduler
import pytorch_lightning as pl
import pandas as pd

class UnifiedTweetPredictor(pl.LightningModule):
    def __init__(self, train_df: pd.DataFrame, val_df: pd.DataFrame, test_df: pd.DataFrame,
                 selected_model: str = 'aubmindlab/bert-base-arabertv02-twitter', n_classes: int = 3,
                 batch_size: int = 8, token_len: int = 128, n_epochs: int = 5):
        super().__init__()
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.token_len = token_len
        self.tokenizer = AutoTokenizer.from_pretrained(selected_model)
        self.bert = AutoModel.from_pretrained(selected_model, return_dict=True)
        self.dropout = nn.Dropout(0.005)  #  dropout rate
        #self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        class_weights = torch.tensor([1.90, 1.0, 1.0])  # Assuming class 0 is underrepresented
#self.criterion = nn.CrossEntropyLoss()
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)
        self.train_f1 = torchmetrics.F1Score(num_classes=n_classes, average='macro',task='MULTICLASS')
        self.val_f1 = torchmetrics.F1Score(num_classes=n_classes, average='macro',task='MULTICLASS')
        self.test_f1 = torchmetrics.F1Score(num_classes=n_classes, average='macro',task='MULTICLASS')
        # Dataset storage
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

    def setup(self, stage=None):
        self.train_dataset = MyDataset(self.train_df, self.tokenizer, max_token_len=self.token_len)
        self.val_dataset = MyDataset(self.val_df, self.tokenizer, max_token_len=self.token_len)
        self.test_dataset = MyDataset(self.test_df, self.tokenizer, max_token_len=self.token_len)


    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask).pooler_output
        output = self.dropout(output)
#output = self.classifier(output)
        output = self.classifier(output)
        output = torch.softmax(output, dim=1)
        # Calculate loss only if labels are provided
        if labels is not None:
            loss = self.criterion(output, labels)
            return loss, output
        else:
        # if labels is not None else None
            return output
#self.val_f1.compute()
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        loss, outputs = self(input_ids, attention_mask, labels)
        preds = torch.argmax(outputs, dim=1)
        self.train_f1.update(preds, labels)
        self.log("train_loss", loss, prog_bar=True)
        self.log('train_f1_score', self.train_f1, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        loss, outputs = self(input_ids, attention_mask, labels)
        preds = torch.argmax(outputs, dim=1)
        self.val_f1.update(preds, labels)
        self.log("val_loss", loss, prog_bar=True)
        self.log('val_f1_score', self.val_f1, on_step=False, on_epoch=True, prog_bar=True)
        #val_f1_score = self.val_f1.compute()
        #print(f"Testing F1 Score (Macro): {val_f1_score:.4f}")

        #self.val_f1.reset()
        return {"loos":loss,
                "f1":self.val_f1}

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        loss, outputs = self(input_ids, attention_mask, labels)
        preds = torch.argmax(outputs, dim=1)
        self.test_f1.update(preds, labels)
        self.log("test_loss", loss, prog_bar=True,logger=True)
        self.log('test_f1_score', self.test_f1, on_step=False, on_epoch=True, prog_bar=True,logger=True)
        #test_f1_score = self.test_f1.compute()
        #self.log('p', preds, on_step=True, on_epoch=True, prog_bar=True)
        test_labels.append(labels)
        test_pred.append(preds)
        probsoft.append(outputs)
        #print(f"Testing F1 Score (Macro): {test_f1_score:.4f}")

        #self.test_f1.reset()
        return {"loos":loss,
                "f1":self.test_f1,
                "predictions": preds,
                "labels": labels}

    def predict_step(self, batch, batch_idx):
       input_ids, attention_mask = batch['input_ids'], batch['attention_mask']
    # Forward pass through the model
       outputs = self(input_ids, attention_mask)

       # Assuming outputs are logits; apply softmax for probabilities
       predictions = torch.softmax(outputs, dim=1)

    # Optionally, return the predicted class indices
       predicted_classes = torch.argmax(predictions, dim=1)


    # Optionally, return the predicted class indices
       #predicted_classes = torch.argmax(predictions, dim=1)

       return {
        #"predictions": predictions
         "predicted_classes": predicted_classes
    }


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=2)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=5e-5)
        scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=self.n_epochs * len(self.train_df))
        return [optimizer], [{'scheduler': scheduler, 'interval': 'step'}]


In [10]:
model = UnifiedTweetPredictor(
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    selected_model='aubmindlab/bert-base-arabertv02-twitter',  # Change as needed try  ... aubmindlab/bert-base-arabert
    n_classes=3,  # Adjust based on the number of label classes
    batch_size=8,  # Can be tuned based on GPU memory
    token_len=128,  # Typical length for BERT; adjust as needed
    n_epochs= 5  # Number of training epochs
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./model_checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    verbose=True,
    mode='min'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Trainer setup
trainer = Trainer(
    max_epochs=model.n_epochs,
    #gpus=1 if torch.cuda.is_available() else 0,  # Use GPUs if available
    callbacks=[checkpoint_callback, early_stopping],
    #progress_bar_refresh_rate=30  # Adjust to control output verbosity

)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [12]:
test_labels = []
test_pred = []
probsoft = []
trainer.fit(model)


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /content/model_checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 135 M 
1 | dropout    | Dropout           | 0     
2 | classifier | Linear            | 2.3 K 
3 | criterion  | CrossEntropyLoss  | 0     
4 | train_f1   | MulticlassF1Score | 0     
5 | val_f1     | MulticlassF1Score | 0     
6 | test_f1    | MulticlassF1Score |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 1182: 'val_loss' reached 0.84039 (best 0.84039), saving model to '/content/model_checkpoints/best-checkpoint-v1.ckpt' as top 1
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 1576: 'val_loss' was not in top 1
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 1970: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [13]:
model.eval()

trainer.test(model)


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_f1_score       │    0.4207362234592438     │
│         test_loss         │    0.9139744639396667     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.9139744639396667, 'test_f1_score': 0.4207362234592438}]

In [21]:
print(test_labels)
print(test_pred )
#print(probsoft)


[tensor([2, 1, 2, 1, 1, 1, 1, 1], device='cuda:0'), tensor([1, 1, 2, 2, 2, 0, 0, 1], device='cuda:0'), tensor([1, 1, 2, 2, 0, 2, 0, 2], device='cuda:0'), tensor([1, 1, 2, 2, 2, 0, 2, 1], device='cuda:0'), tensor([2, 2, 2, 2, 0, 1, 2, 1], device='cuda:0'), tensor([2, 2, 2, 1, 2, 2, 0, 1], device='cuda:0'), tensor([1, 1, 1, 1, 1, 1, 2, 0], device='cuda:0'), tensor([2, 1, 2, 1, 1, 1, 1, 1], device='cuda:0'), tensor([1, 1, 1, 1, 2, 2, 1, 1], device='cuda:0'), tensor([1, 2, 1, 2, 2, 1, 2, 0], device='cuda:0'), tensor([1, 1, 1, 1, 1, 2, 2, 2], device='cuda:0'), tensor([2, 1, 1, 0, 1, 1, 2, 0], device='cuda:0'), tensor([1, 2, 0, 2, 1, 0, 0, 1], device='cuda:0'), tensor([2, 1, 1, 1, 1, 2, 1, 1], device='cuda:0'), tensor([1, 2, 2, 2, 0, 1, 1, 0], device='cuda:0'), tensor([1, 1, 2, 1, 1, 1, 0, 2], device='cuda:0'), tensor([1, 0, 1, 2, 1, 1, 1, 2], device='cuda:0'), tensor([0, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), tensor([1, 1, 0, 1, 2, 0, 0, 0], device='cuda:0'), tensor([1, 1, 2, 1, 1, 2, 1, 1

In [19]:
from torch.utils.data import Dataset
import torch

class NewTestDataset(Dataset):
    def __init__(self, data, tokenizer, max_token_len=128):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Extract the text from the DataFrame
        text = self.data[index]

        # Perform tokenization (assuming the preprocessing step is included in the tokenizer)
        # If separate preprocessing is needed, apply it before tokenization
        # e.g., text = preprocess_function(text)
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,    # Add '[CLS]' and '[SEP]'
            max_length=self.max_token_len,  # Pad & truncate all sentences.
            padding='max_length',
            truncation=True,
            return_attention_mask=True,  # Construct attention masks.
            return_tensors='pt'         # Return PyTorch tensors.
        )

        return dict(
           text=text,
      input_ids=encoding["input_ids"].flatten(), ##we use flatten to remove X dimension
      attention_mask=encoding["attention_mask"].flatten()# Flatten the tensor
        )




In [20]:
# Example usage
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabertv02-twitter')
df_new_data = pd.read_csv("Mawqif_AllTargets_Blind Test.csv")  # Your DataFrame containing new texts
new_test_dataset = NewTestDataset(df_new_data['text'].tolist(), tokenizer, max_token_len=128)
new_test_loader = DataLoader(new_test_dataset, batch_size=8, shuffle=False)

FileNotFoundError: [Errno 2] No such file or directory: 'Mawqif_AllTargets_Blind Test.csv'

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [ ]:
!nvidia-smi

In [ ]:
# Predict the labels for the blind test data

model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
from pytorch_lightning import Trainer

trainer = Trainer(accelerator="gpu" if torch.cuda.is_available() else "cpu")
predictions = trainer.predict(model, dataloaders=new_test_loader)


In [ ]:
import torch

# Assume `predictions` is the output from `trainer.predict()`
all_predictions = []
for batch in predictions:
    all_predictions.extend(batch['predicted_classes'].tolist())

# Now `all_predictions` is a single list of all predictions
print(all_predictions)


In [ ]:
stance_mapping = {'Favor': 1, 'Against': 2, 'None': 0}
inverse_stance_mapping = {v: k for k, v in stance_mapping.items()}


In [ ]:
labeled_predictions = [inverse_stance_mapping[pred] for pred in all_predictions]
print(labeled_predictions)

In [ ]:
# Save or print predictions
df_new_data['predicted_stance'] = labeled_predictions
print(df_new_data )

In [ ]:
df_new_data.to_csv('Mawqif_AllTargets_Blind_Test_Predictions.csv', index=False)